In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
import json

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini",max_retries=1)

In [4]:
@tool
def count_emails(last_n_days: int) -> int:
    """Multiply two integers together."""
    return last_n_days * 2


@tool
def send_email(message: str, recipient: str) -> str:
    "Send email message to the recipient"
    return f"Successfully sent email:\n{message}\n to {recipient}."

In [5]:
tools=[count_emails,send_email]
llmWithTools=llmGPT.bind_tools(tools=tools)

In [6]:
def toolIdentifier(msg: AIMessage) -> Runnable:
    toolMap={tool.name:tool for tool in tools}
    for toolInfo in msg.tool_calls:
        toolInfo['output']=toolMap[toolInfo['name']].invoke(input=toolInfo['args'])
    return msg.tool_calls

In [7]:
chain=llmWithTools|toolIdentifier

In [8]:
question1="How many emails did I get in the last 5 days?"

In [9]:
chain.invoke(input=question1)

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_KOk3e2wJDpHO8zi2WYPNCtsQ',
  'type': 'tool_call',
  'output': 10}]

In [10]:
question2="Send an Email to Mr. Ritish Adhikari stating that you will be late?"

In [11]:
chain.invoke(input=question2)

[{'name': 'send_email',
  'args': {'message': 'Dear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be running late. I apologize for any inconvenience this may cause and appreciate your understanding.\n\nBest regards,\n\n[Your Name]',
   'recipient': 'ritish.adhikari@example.com'},
  'id': 'call_RxcWqRAzlQ6JZIADLj5HPhfX',
  'type': 'tool_call',
  'output': 'Successfully sent email:\nDear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be running late. I apologize for any inconvenience this may cause and appreciate your understanding.\n\nBest regards,\n\n[Your Name]\n to ritish.adhikari@example.com.'}]

<h3> Adding of Human Approval </h3>

In [12]:
toolCall=llmWithTools.invoke(input=question2).tool_calls[0]
toolCall

{'name': 'send_email',
 'args': {'message': 'Dear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be running late. I apologize for any inconvenience this may cause and will keep you updated on my arrival time.\n\nThank you for your understanding.\n\nBest regards,\n\n[Your Name]',
  'recipient': 'ritish.adhikari@example.com'},
 'id': 'call_aPtpftFS1kRdilA0YX9Oi5FB',
 'type': 'tool_call'}

In [13]:
print(json.dumps(obj=toolCall,indent=2))

{
  "name": "send_email",
  "args": {
    "message": "Dear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be running late. I apologize for any inconvenience this may cause and will keep you updated on my arrival time.\n\nThank you for your understanding.\n\nBest regards,\n\n[Your Name]",
    "recipient": "ritish.adhikari@example.com"
  },
  "id": "call_aPtpftFS1kRdilA0YX9Oi5FB",
  "type": "tool_call"
}


In [14]:
print("".join(json.dumps(obj=toolcall, indent=2) for toolcall in llmWithTools.invoke(input=question2).tool_calls))

{
  "name": "send_email",
  "args": {
    "message": "Dear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be running late. I apologize for any inconvenience this may cause and appreciate your understanding.\n\nThank you for your patience.\n\nBest regards,\n\n[Your Name]",
    "recipient": "ritish.adhikari@example.com"
  },
  "id": "call_AkPjvYJBVb6lqZ1rCa29KxUc",
  "type": "tool_call"
}


In [16]:
def humanApproval(msg: AIMessage) -> Runnable:
    toolStrs="".join(json.dumps(obj=toolcall, indent=2) for toolcall in msg.tool_calls)

    inputMessage=(
        f"Do you approve of the following tool invocations \n\n {toolStrs}\n\n"
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a No")

    resp=input(inputMessage)
    if resp.lower() not in ['y','yes']:
        raise ValueError(f"Tool Invocation is Not approved: \n\n {toolStrs}")
    else:
        return msg

In [17]:
chain=llmWithTools|humanApproval|toolIdentifier

In [18]:
chain.invoke(input=question2)

[{'name': 'send_email',
  'args': {'message': 'Dear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be late. Thank you for your understanding.\n\nBest regards,\n\n[Your Name]',
   'recipient': 'ritish.adhikari@example.com'},
  'id': 'call_nfp6tV38MDNIlytK88M05ZqF',
  'type': 'tool_call',
  'output': 'Successfully sent email:\nDear Mr. Ritish Adhikari,\n\nI hope this message finds you well. I wanted to inform you that I will be late. Thank you for your understanding.\n\nBest regards,\n\n[Your Name]\n to ritish.adhikari@example.com.'}]

In [19]:
chain.invoke(input=question1)

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_8PLYJ9nqQNIjXc53Z3zNAqvx',
  'type': 'tool_call',
  'output': 10}]